In [1]:
import pickle
import os
import numpy as np
import pandas as pd
import json

In [2]:
df = pd.read_pickle("BNLCosmic4_DF.pkl")

In [3]:
# Generate the header section
header = {
    "version": "2.2.0",
    "experiment": f"ZDC Prototype",
    "energy_unit": "ADC", # Select "eV", "KeV", "MeV", "GeV", or "TeV"
    "color_bar": "Lin", # Select "Lin" or "Log"
    "length_unit": "cm",
    "time_scale": 10,
    "particles": [
        {
        "angle_rad": [0.0 , 0.0],       
        "size": float(10),
        "color_rgba": [float(1), float(0), float(0), float(1)]
        }
    ],
    "tracker_settings": {
        "B_field_T": float(0),
        "tracker_boundary": [float(1), float(1), float(1)]
    }
}

# Main data structure
data = {
    "header": header,
    "events": []
}

In [4]:
# Group the DataFrame by event number
grouped = df.groupby('event')

events = []
maxE = df['HG'].max()  # or a manually set constant if needed

for evt_num, group in grouped:
    info_text = f"Event #{evt_num}"
    event = {
        "event_data": {
            "info_text": info_text,
            "energy_scale": [0, float(maxE)]
        },
        "hits": [],
        "clusters": [],
        "tracks": [],
        "jets": [],
        "blocks": []
    }
    
    blocks = []
    for _, row in group.iterrows():
        energy = row['HG_ped_corr']
        if energy <= 0:
            continue  # skip zero or negative energies
        
        x, y, z = row['x'], row['y'], row['z']
        normalized_energy = energy / maxE
        red, green, blue, alpha = normalized_energy, 0, 1 - normalized_energy, normalized_energy
        c = 299.792
        time = z / c

        block = {
            "position": [float(x), float(y), float(z)],
            "time_ns": float(time),
            "size": [48.8, 48.8, 4],
            "color_rgba": [float(red), float(green), float(blue), float(alpha)]
        }
        blocks.append(block)
    
    event["blocks"] = blocks
    events.append(event)

# Final output dictionary
data = {
    "header": header,
    "events": events
}




In [6]:
eventPath = f'BNLCosmic4.json'
with open(eventPath, 'w') as outfile:
    json.dump(data, outfile, indent=4)